In [37]:
import warnings
warnings.filterwarnings("ignore")

from fbprophet import Prophet
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

from scipy import stats
import statsmodels.api as sm
from itertools import product
from datetime import timedelta

import pandas as pd
pd.options.display.max_columns = 999
import numpy as np

from sklearn.metrics import mean_squared_error
#RMSE = mean_squared_error(y, y_pred)**0.5

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []

    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)

    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

In [9]:
# 1_2017-06-23 \ 1_2017-06-29
#15621 нет в dataset

## Подготовка данных

In [34]:
df_price = pd.read_csv('data/player_price.csv')
df_stats = pd.read_csv('data/player_stats.csv')
sample_subm = pd.read_csv('data/kaggle_sample_submission.csv')

sample_subm["player_id"] = sample_subm["id"].apply(lambda x: int(x.split("_")[0]))
sample_subm["Date"] = sample_subm["id"].apply(lambda x: np.datetime64(x.split("_")[1]))

#player_id=df_price.groupby(['player_id'])['xbox_price'].count()
player_id=sample_subm.groupby(['player_id'])['price'].count()

#df_price=df_price[df_price.player_id<=18320]
df_price['Date'] = pd.to_datetime(df_price['timestamp'], unit='ms')
df_price.drop(['timestamp'], axis=1, inplace=True)
df_price.drop(['ps_price'],axis = 1, inplace=True)
df_price.rename(columns={'xbox_price':'price'}, inplace = True)

df_merge=df_stats[["player_id"]].merge(df_price, on=['player_id'], how='inner')

df_merge.shape

(980252, 4)

In [35]:
df_player=df_merge[df_merge.player_id==1]
df_player.head()

,player_id,price,player_name,Date
0,1,0,Pelé,2016-09-23
1,1,10000000,Pelé,2016-09-24
2,1,0,Pelé,2016-09-25
3,1,0,Pelé,2016-09-26
4,1,10000000,Pelé,2016-09-27


In [51]:
df=df_player[['Date','price']]
df=df.groupby('Date')[['price']].mean()
df=df.reset_index()
#df.columns = ['ds', 'y']
df=df.set_index('Date')
#df=df.replace(0, int(df.mean()))
df = df[df.price!=0]

print(df.shape)
df.head()

(269, 1)


,price
Date,
2016-09-24,10000000
2016-09-27,10000000
2016-09-29,9500000
2016-09-30,8428000
2016-10-01,8459091


In [39]:
lr=LinearRegression(n_jobs=-1)

In [50]:
lr.fit(df.Date.values, df.price.values)

ValueError: Found input variables with inconsistent numbers of samples: [1, 269]

In [127]:
%%time
k=1
for i in player_id.index[732:len(player_id.index)]:#player_id.index:#range(1,6):
    print('==================================')
    print('id={}'.format(i))
    
    df_player=df_merge[df_merge.player_id==i]
        
    df=df_player[['Date','price']]
    df=df.groupby('Date')[['price']].mean()
    df=df.reset_index()
    df.columns = ['ds', 'y']
    df=df.replace(0,int(df.mean()))
    
    train_df=df
    
    m = Prophet()
    m.fit(train_df)
    
    future = m.make_future_dataframe(periods=7)
    forecast = m.predict(future)
    
    cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df.set_index('ds'))
    
    qwe=cmp_df.yhat[-7:]    
    print(qwe)
    
    pp=pd.DataFrame(columns=['id','price'])
    pp['id']=str(i)+'_'+qwe.index.astype(str)
    pp.price=qwe.values
    pp.price=pp.price.astype(int)

    if k==0:
        pp.to_csv('prediction_fbpophet.csv',sep=',',header=True,index=False)
        k=1
    else:
        with open('prediction_fbpophet.csv', 'a') as f:
            pp.to_csv(f, header=False,index=False,sep=',')

id=15621
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    16343.322412
2017-06-24    16615.223397
2017-06-25    16821.018940
2017-06-26    16956.366858
2017-06-27    17034.962847
2017-06-28    17052.268128
2017-06-29    16027.012197
Name: yhat, dtype: float64
id=15622
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    45898.951769
2017-06-24    44479.936077
2017-06-25    45145.533419
2017-06-26    42858.267165
2017-06-27    41655.052075
2017-06-28    42979.341833
2017-06-29    40957.052004
Name: yhat, dtype: float64
id=15623
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    97262.896507
2017-06-24    93658.170291
2017-06-25    90686.153156
2017-06-26    89493.935127
2017-06-27    87604.869573
2017-06-28    87285.178343
2017-06-29    86098.747154
Name: yhat, dtype: float64
id=15624
Disabling yearly seasonality. Run prophe

ds
2017-06-23    20205.775595
2017-06-24    17605.468359
2017-06-25    17632.635890
2017-06-26    17017.563753
2017-06-27    17494.722067
2017-06-28    17528.451001
2017-06-29    17545.951327
Name: yhat, dtype: float64
id=15965
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    28925.499936
2017-06-24    25957.152983
2017-06-25    24974.008556
2017-06-26    23571.962319
2017-06-27    23498.418646
2017-06-28    23483.853138
2017-06-29    23630.066735
Name: yhat, dtype: float64
id=15966
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    19347.956215
2017-06-24    17539.936700
2017-06-25    17771.924209
2017-06-26    17607.420890
2017-06-27    18157.266446
2017-06-28    18172.487059
2017-06-29    18091.773490
Name: yhat, dtype: float64
id=15967
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    132036.740127
2017-06-24    1247

ds
2017-06-23    8324.007876
2017-06-24    7955.140430
2017-06-25    8132.665357
2017-06-26    8470.049595
2017-06-27    8463.647313
2017-06-28    8347.921544
2017-06-29    8368.123376
Name: yhat, dtype: float64
id=16042
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    7168.439617
2017-06-24    7233.186116
2017-06-25    7666.782574
2017-06-26    6706.117615
2017-06-27    7121.148998
2017-06-28    7007.765922
2017-06-29    7335.925823
Name: yhat, dtype: float64
id=16111
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    7184.469059
2017-06-24    6944.552725
2017-06-25    6869.986673
2017-06-26    7143.722847
2017-06-27    7047.593134
2017-06-28    7426.936148
2017-06-29    7124.490183
Name: yhat, dtype: float64
id=16203
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    5150.567607
2017-06-24    4346.872260
2017-06-25    4

ds
2017-06-23    5396.648081
2017-06-24    5478.962169
2017-06-25    5692.531828
2017-06-26    5936.381721
2017-06-27    6300.600969
2017-06-28    6303.836274
2017-06-29    5942.621608
Name: yhat, dtype: float64
id=16424
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    6032.064758
2017-06-24    6245.030639
2017-06-25    6173.840848
2017-06-26    6445.334467
2017-06-27    6128.237659
2017-06-28    6095.269695
2017-06-29    6146.108797
Name: yhat, dtype: float64
id=16447
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    12151.189526
2017-06-24    12164.057733
2017-06-25    12157.612415
2017-06-26    12240.217636
2017-06-27    12078.177328
2017-06-28    12226.444733
2017-06-29    12210.253216
Name: yhat, dtype: float64
id=16448
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    14232.817573
2017-06-24    14838.329934
2017-0

ds
2017-06-23    23932.073523
2017-06-24    23849.036859
2017-06-25    23780.380478
2017-06-26    23613.979023
2017-06-27    24195.218986
2017-06-28    23948.349938
2017-06-29    24911.393349
Name: yhat, dtype: float64
id=16480
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    107737.090593
2017-06-24     89280.130673
2017-06-25     86832.406664
2017-06-26     85033.728060
2017-06-27     83889.458110
2017-06-28    112744.176891
2017-06-29    104737.578247
Name: yhat, dtype: float64
id=16481
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    18153.494135
2017-06-24    16446.236822
2017-06-25    16629.626014
2017-06-26    16323.440917
2017-06-27    16298.231983
2017-06-28    16615.579287
2017-06-29    16917.001198
Name: yhat, dtype: float64
id=16482
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    21258.119353
2017-06-24  

...

ds
2017-06-23    1.822772e+06
2017-06-24    1.797881e+06
2017-06-25    1.822823e+06
2017-06-26    1.844640e+06
2017-06-27    1.858292e+06
2017-06-28    1.915767e+06
2017-06-29    1.904881e+06
Name: yhat, dtype: float64
id=18153
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    33998.928093
2017-06-24    31152.241494
2017-06-25    31080.318528
2017-06-26    31502.012101
2017-06-27    29230.416850
2017-06-28    40856.924345
2017-06-29    37646.588628
Name: yhat, dtype: float64
id=18154
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    618525.989525
2017-06-24    575683.815506
2017-06-25    585938.489767
2017-06-26    587269.924566
2017-06-27    580237.651415
2017-06-28    741623.218592
2017-06-29    758880.169833
Name: yhat, dtype: float64
id=18155
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    20990.328853
2017-06-24  

ds
2017-06-23    355107.689955
2017-06-24    358339.333893
2017-06-25    351538.654685
2017-06-26    348348.190687
2017-06-27    352729.195784
2017-06-28    349157.737513
2017-06-29    352406.792254
Name: yhat, dtype: float64
id=18185
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    66627.757657
2017-06-24    65471.521671
2017-06-25    66995.551952
2017-06-26    65613.181480
2017-06-27    65841.649121
2017-06-28    65531.546472
2017-06-29    67070.808105
Name: yhat, dtype: float64
id=18187
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    25264.894686
2017-06-24    25260.481231
2017-06-25    26603.374313
2017-06-26    25846.533464
2017-06-27    25571.987124
2017-06-28    25309.514159
2017-06-29    24228.713884
Name: yhat, dtype: float64
id=18189
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    11800.557519
2017-06-24  

ds
2017-06-23    122808.282588
2017-06-24    120766.586238
2017-06-25    125921.450634
2017-06-26    123593.819866
2017-06-27    123183.914382
2017-06-28    136203.591598
2017-06-29    126531.314964
Name: yhat, dtype: float64
id=18262
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    18319.844951
2017-06-24    18710.917965
2017-06-25    19253.410108
2017-06-26    18690.707318
2017-06-27    18500.053898
2017-06-28    18144.966247
2017-06-29    18243.755457
Name: yhat, dtype: float64
id=18264
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    243073.785005
2017-06-24    243473.780996
2017-06-25    252056.570269
2017-06-26    245601.009615
2017-06-27    242507.763919
2017-06-28    244263.871287
2017-06-29    250293.767076
Name: yhat, dtype: float64
id=18265
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23     99431.753973
2017

ds
2017-06-23    16741.531604
2017-06-24    15739.653921
2017-06-25    15234.457687
2017-06-26    16882.585034
2017-06-27    18757.266381
2017-06-28    19402.445854
2017-06-29    18043.222910
Name: yhat, dtype: float64
id=18301
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    120076.275775
2017-06-24    112241.922703
2017-06-25    110993.260067
2017-06-26    108318.526394
2017-06-27    108245.344782
2017-06-28    178824.490513
2017-06-29    147333.361143
Name: yhat, dtype: float64
id=18302
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    110698.019586
2017-06-24    104665.705261
2017-06-25    106200.261686
2017-06-26    104820.467188
2017-06-27    104159.387119
2017-06-28    117592.951421
2017-06-29    118523.591902
Name: yhat, dtype: float64
id=18303
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    24484.093154
2017-

ds
2017-06-23    471247.922204
2017-06-24    424011.351201
2017-06-25    426352.587768
2017-06-26    416783.950050
2017-06-27    413531.573023
2017-06-28    410084.223112
2017-06-29    415678.224522
Name: yhat, dtype: float64
id=18330
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    268687.259198
2017-06-24    263393.692462
2017-06-25    257727.987189
2017-06-26    251335.179918
2017-06-27    256536.652079
2017-06-28    256266.715975
2017-06-29    258984.114914
Name: yhat, dtype: float64
id=18331
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    511531.174456
2017-06-24    490021.026024
2017-06-25    488098.352488
2017-06-26    482444.560910
2017-06-27    484219.784193
2017-06-28    486547.368654
2017-06-29    485780.868109
Name: yhat, dtype: float64
id=18332
Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
ds
2017-06-23    20967.93342